# UK VAT Threshold Simulation

This notebook demonstrates the behavioral effects of VAT threshold changes using elasticities from academic literature.

## Key Parameters from Literature

- **Turnover elasticity**: 0.09-0.18 (Liu et al. 2019)
- **Growth slowdown**: ~1 percentage point near threshold (IMF 2024)
- **Bunching behavior**: 15-20% of firms near threshold bunch below it

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd()))
from src.vat_simulation import VATSimulator, VATPolicy

# Set random seed for reproducibility
np.random.seed(42)
print("Libraries loaded successfully")

## 1. Load Synthetic Firm Data

We use synthetic data representing UK firms with realistic turnover distributions.

In [ ]:
# Load synthetic firms
firms_df = pd.read_csv('analysis/synthetic_firms_quick.csv')
print(f"Loaded {len(firms_df):,} synthetic firms\n")

# Show distribution
print("Turnover Distribution:")
print(f"  Below £85k: {(firms_df['annual_turnover'] < 85000).sum():,} firms")
print(f"  £85k-£90k: {((firms_df['annual_turnover'] >= 85000) & (firms_df['annual_turnover'] < 90000)).sum():,} firms")
print(f"  Above £90k: {(firms_df['annual_turnover'] >= 90000).sum():,} firms")

# Display sample
firms_df.head()

## 2. Understanding the VAT Notch Effect

The VAT threshold creates a "notch" - a discontinuous jump in tax liability. Firms just below £90k pay no VAT, while those just above must register and charge 20% VAT on their sales.

This creates two behavioral responses:
1. **Bunching**: Firms deliberately stay below the threshold
2. **Growth slowdown**: Reduced business activity to avoid crossing

In [ ]:
# Visualize original turnover distribution near threshold
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Full distribution
ax1.hist(firms_df['annual_turnover'] / 1000, bins=100, alpha=0.7, edgecolor='black')
ax1.axvline(90, color='red', linestyle='--', linewidth=2, label='VAT Threshold (£90k)')
ax1.set_xlabel('Annual Turnover (£k)')
ax1.set_ylabel('Number of Firms')
ax1.set_title('Full Turnover Distribution')
ax1.set_xlim(0, 200)
ax1.legend()

# Zoomed view near threshold
near_threshold = firms_df[(firms_df['annual_turnover'] >= 70000) & 
                          (firms_df['annual_turnover'] <= 110000)]
ax2.hist(near_threshold['annual_turnover'] / 1000, bins=40, alpha=0.7, 
         color='orange', edgecolor='black')
ax2.axvline(90, color='red', linestyle='--', linewidth=2, label='VAT Threshold')
ax2.axvspan(85, 90, alpha=0.2, color='yellow', label='Typical bunching zone')
ax2.set_xlabel('Annual Turnover (£k)')
ax2.set_ylabel('Number of Firms')
ax2.set_title('Distribution Near Threshold (£70k-£110k)')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"\nNotice the concentration of firms just below £90k - this is 'bunching'")

## 3. Initialize VAT Simulator with Literature-Based Parameters

In [ ]:
# Create simulator with parameters from academic literature
simulator = VATSimulator(
    firms=firms_df,
    elasticity_turnover=0.14,  # Mid-point from Liu et al. (0.09-0.18)
    bunching_rate=0.15,        # 15% of firms near threshold will bunch
    growth_slowdown=0.01       # 1 percentage point growth reduction
)

print("Simulator initialized with:")
print(f"  Elasticity: 0.14 (firms reduce turnover by 14% of VAT burden)")
print(f"  Bunching rate: 15% (of firms near threshold)")
print(f"  Growth slowdown: 1pp (for firms just above threshold)")

## 4. Simulate Different Policy Scenarios

In [ ]:
# Define policy scenarios
policies = {
    'Current (£90k)': VATPolicy(threshold=90000, standard_rate=0.20),
    'Lower (£85k)': VATPolicy(threshold=85000, standard_rate=0.20),
    'Higher (£100k)': VATPolicy(threshold=100000, standard_rate=0.20),
    'With Marginal Rates': VATPolicy(
        threshold=90000,
        standard_rate=0.20,
        reduced_rates={
            (30000, 60000): 0.10,  # 10% for firms £30k-£60k
            (60000, 90000): 0.15   # 15% for firms £60k-£90k
        }
    )
}

# Run simulations
results = {}
for name, policy in policies.items():
    results[name] = simulator.simulate(policy)
    print(f"\n{name}:")
    print(f"  Revenue: £{results[name]['vat_revenue']/1e9:.2f}B")
    print(f"  Registered firms: {results[name]['num_registered']:,}")
    print(f"  Bunching firms: {results[name]['firms_bunching']:,}")
    print(f"  Behavioral cost: £{-results[name]['behavioral_adjustment']/1e6:.1f}M")

## 5. Visualize Behavioral Responses

In [ ]:
# Compare distributions before and after behavioral response
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

scenarios = ['Current (£90k)', 'Lower (£85k)']
thresholds = [90000, 85000]

for idx, (scenario, threshold) in enumerate(zip(scenarios, thresholds)):
    # Original distribution
    ax = axes[idx, 0]
    region = firms_df[(firms_df['annual_turnover'] >= threshold - 20000) & 
                      (firms_df['annual_turnover'] <= threshold + 20000)]
    ax.hist(region['annual_turnover'] / 1000, bins=40, alpha=0.7, 
            color='blue', edgecolor='black', label='Original')
    ax.axvline(threshold/1000, color='red', linestyle='--', linewidth=2)
    ax.set_xlabel('Annual Turnover (£k)')
    ax.set_ylabel('Number of Firms')
    ax.set_title(f'{scenario} - Original Distribution')
    ax.legend()
    
    # After behavioral response
    ax = axes[idx, 1]
    adjusted = results[scenario]['adjusted_firms']
    region_adj = adjusted[(adjusted['annual_turnover'] >= threshold - 20000) & 
                         (adjusted['annual_turnover'] <= threshold + 20000)]
    ax.hist(region_adj['annual_turnover'] / 1000, bins=40, alpha=0.7, 
            color='green', edgecolor='black', label='After response')
    ax.axvline(threshold/1000, color='red', linestyle='--', linewidth=2)
    ax.axvspan(threshold*0.98/1000, threshold/1000, alpha=0.3, color='yellow', 
               label='Bunching zone')
    ax.set_xlabel('Annual Turnover (£k)')
    ax.set_ylabel('Number of Firms')
    ax.set_title(f'{scenario} - After Behavioral Response')
    ax.legend()

plt.tight_layout()
plt.show()

## 6. Policy Comparison Summary

In [ ]:
# Create comparison table
comparison_data = []
for name, result in results.items():
    policy = result['policy']
    comparison_data.append({
        'Policy': name,
        'Threshold': f"£{policy.threshold:,.0f}",
        'Revenue (£B)': f"{result['vat_revenue']/1e9:.2f}",
        'Behavioral Cost (£M)': f"{-result['behavioral_adjustment']/1e6:.1f}",
        'Efficiency Loss': f"{-result['behavioral_adjustment']/result['baseline_revenue']*100:.1f}%",
        'Registered Firms': f"{result['num_registered']:,}",
        'Bunching Firms': f"{result['firms_bunching']:,}"
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nPolicy Comparison Table:")
print("="*80)
print(comparison_df.to_string(index=False))

## 7. Key Insights

### The VAT Notch Problem
- Firms bunch below the threshold to avoid VAT registration
- This creates economic inefficiency as firms limit their growth
- The behavioral response reduces VAT revenue by millions

### Policy Trade-offs
1. **Lower threshold (£85k)**: More revenue but greater distortions
2. **Higher threshold (£100k)**: Less distortion but lower revenue
3. **Marginal rates**: Could smooth the transition and reduce bunching

### Elasticity Effects
- With elasticity of 0.14, firms reduce turnover by ~2.8% when facing 20% VAT
- This behavioral response costs the government significant revenue

In [ ]:
# Calculate specific insights
current = results['Current (£90k)']
lower = results['Lower (£85k)']
marginal = results['With Marginal Rates']

print("\nQUANTIFIED INSIGHTS:")
print("="*50)
print(f"\n1. BUNCHING EFFECT:")
print(f"   {current['firms_bunching']:,} firms bunch below £90k")
print(f"   This is {current['firms_bunching']/len(firms_df)*100:.1f}% of all firms")

print(f"\n2. REVENUE IMPACT OF THRESHOLD CHANGE:")
print(f"   Lowering to £85k: +£{(lower['vat_revenue']-current['vat_revenue'])/1e9:.2f}B revenue")
print(f"   But increases bunching to {lower['firms_bunching']:,} firms")

print(f"\n3. EFFICIENCY LOSS:")
print(f"   Current policy loses {-current['behavioral_adjustment']/current['baseline_revenue']*100:.1f}% to behavioral responses")
print(f"   This equals £{-current['behavioral_adjustment']/1e6:.1f}M in lost revenue")

print(f"\n4. MARGINAL RATES OPTION:")
print(f"   Changes revenue by £{(marginal['vat_revenue']-current['vat_revenue'])/1e6:.1f}M")
print(f"   Could help smooth the threshold discontinuity")